# ImageNet Training with ResNet50 from Scratch

This notebook implements ImageNet training for ResNet50 from scratch, targeting 75% top-1 accuracy within a $25 budget.

## Key Features:
- ResNet50 implementation from scratch
- Optimized for budget constraints
- Mixed precision training
- Data augmentation strategies
- Model checkpointing and evaluation


In [ ]:
# Install required packages
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install timm
%pip install wandb
%pip install accelerate
%pip install transformers
%pip install datasets
%pip install huggingface_hub


In [ ]:
# =============================================================================
# 🚀 EASY SWITCH: TESTING vs PRODUCTION MODE
# =============================================================================
# 
# TO RUN IN COLAB (TESTING MODE):
#   1. Set TESTING_MODE = True
#   2. Set PRODUCTION_MODE = False
#
# TO RUN IN PRODUCTION (IMAGENET):
#   1. Set TESTING_MODE = False  
#   2. Set PRODUCTION_MODE = True
#
# =============================================================================

# 🧪 TESTING MODE (Colab/Development)
TESTING_MODE = True
PRODUCTION_MODE = False

# 🏭 PRODUCTION MODE (ImageNet Training)
# TESTING_MODE = False
# PRODUCTION_MODE = True

# =============================================================================
# 🚀 QUANTIZATION OPTIONS FOR SPEED & BUDGET OPTIMIZATION
# =============================================================================

# QUANTIZATION_MODE options:
# - "none": No quantization (baseline)
# - "fp16": Mixed precision (already enabled)
# - "int8": 8-bit quantization (faster, smaller)
# - "dynamic": Dynamic quantization (runtime)
# - "qat": Quantization Aware Training (best accuracy)

QUANTIZATION_MODE = "fp16"  # Change this to experiment with different quantization

# Advanced quantization settings
QUANTIZATION_CONFIG = {
    "fp16": {
        "description": "Mixed Precision (FP16) - 2x speed, 50% memory",
        "speed_boost": "2x",
        "memory_saving": "50%",
        "accuracy_loss": "0-1%",
        "cost_reduction": "30-40%"
    },
    "int8": {
        "description": "8-bit Quantization - 3x speed, 75% memory",
        "speed_boost": "3x", 
        "memory_saving": "75%",
        "accuracy_loss": "1-3%",
        "cost_reduction": "50-60%"
    },
    "dynamic": {
        "description": "Dynamic Quantization - 2.5x speed, 60% memory",
        "speed_boost": "2.5x",
        "memory_saving": "60%", 
        "accuracy_loss": "0.5-2%",
        "cost_reduction": "40-50%"
    },
    "qat": {
        "description": "Quantization Aware Training - 2.5x speed, 60% memory",
        "speed_boost": "2.5x",
        "memory_saving": "60%",
        "accuracy_loss": "0-1%",
        "cost_reduction": "40-50%"
    }
}

# =============================================================================
# 📊 CONFIGURATION BASED ON MODE
# =============================================================================

if TESTING_MODE:
    print("🧪 RUNNING IN TESTING MODE (CIFAR-100)")
    print("   - Dataset: CIFAR-100 (100 classes)")
    print("   - Epochs: 5 (reduced for testing)")
    print("   - Batch Size: 16 (memory efficient)")
    print("   - Target Accuracy: 80%")
    print("   - Wandb: Disabled")
    
elif PRODUCTION_MODE:
    print("🏭 RUNNING IN PRODUCTION MODE (ImageNet-1K)")
    print("   - Dataset: ImageNet-1K (1000 classes)")
    print("   - Epochs: 90 (full training)")
    print("   - Batch Size: 64 (optimized)")
    print("   - Target Accuracy: 75%")
    print("   - Wandb: Enabled")
    
else:
    raise ValueError("Please set either TESTING_MODE=True or PRODUCTION_MODE=True")

print("=" * 60)


In [ ]:
# =============================================================================
# 🔧 QUANTIZATION IMPLEMENTATION FUNCTIONS
# =============================================================================

import torch.quantization as quant
from torch.quantization import QuantStub, DeQuantStub
import torch.nn.utils.prune as prune

def apply_quantization(model, quantization_mode, device):
    """
    Apply different quantization strategies to the model
    """
    print(f"🔧 Applying {quantization_mode} quantization...")
    
    if quantization_mode == "none":
        print("   → No quantization applied (baseline)")
        return model
        
    elif quantization_mode == "fp16":
        print("   → Using mixed precision (FP16) - 2x speed boost")
        # Mixed precision is handled by Accelerator
        return model
        
    elif quantization_mode == "int8":
        print("   → Applying 8-bit quantization - 3x speed boost")
        # Set model to evaluation mode for quantization
        model.eval()
        
        # Configure quantization
        model.qconfig = quant.get_default_qconfig('fbgemm')
        
        # Prepare model for quantization
        model_prepared = quant.prepare(model)
        
        # Calibrate with dummy data (in practice, use real data)
        dummy_input = torch.randn(1, 3, 224, 224).to(device)
        with torch.no_grad():
            model_prepared(dummy_input)
        
        # Convert to quantized model
        model_quantized = quant.convert(model_prepared)
        print("   → Model quantized to INT8")
        return model_quantized
        
    elif quantization_mode == "dynamic":
        print("   → Applying dynamic quantization - 2.5x speed boost")
        # Dynamic quantization (weights quantized, activations in FP32)
        model_quantized = quant.quantize_dynamic(
            model, 
            {torch.nn.Linear, torch.nn.Conv2d}, 
            dtype=torch.qint8
        )
        print("   → Model dynamically quantized")
        return model_quantized
        
    elif quantization_mode == "qat":
        print("   → Setting up Quantization Aware Training - 2.5x speed boost")
        # QAT requires special setup - we'll configure it for training
        model.qconfig = quant.get_default_qat_qconfig('fbgemm')
        model_prepared = quant.prepare_qat(model)
        print("   → Model prepared for QAT")
        return model_prepared
        
    else:
        raise ValueError(f"Unknown quantization mode: {quantization_mode}")

def get_quantization_info(model, quantization_mode):
    """
    Get information about model size and performance with quantization
    """
    if quantization_mode == "none":
        return {
            "model_size_mb": sum(p.numel() for p in model.parameters()) * 4 / 1e6,
            "parameters": sum(p.numel() for p in model.parameters()),
            "speed_boost": "1x",
            "memory_saving": "0%"
        }
    
    # Calculate quantized model size
    total_params = sum(p.numel() for p in model.parameters())
    
    if quantization_mode == "fp16":
        model_size = total_params * 2 / 1e6  # 2 bytes per parameter
        return {
            "model_size_mb": model_size,
            "parameters": total_params,
            "speed_boost": "2x",
            "memory_saving": "50%"
        }
    elif quantization_mode in ["int8", "dynamic", "qat"]:
        model_size = total_params * 1 / 1e6  # 1 byte per parameter
        return {
            "model_size_mb": model_size,
            "parameters": total_params,
            "speed_boost": "3x" if quantization_mode == "int8" else "2.5x",
            "memory_saving": "75%" if quantization_mode == "int8" else "60%"
        }

def print_quantization_summary(quantization_mode, config_info):
    """
    Print a summary of quantization benefits
    """
    print("\n" + "="*60)
    print("🚀 QUANTIZATION BENEFITS SUMMARY")
    print("="*60)
    
    if quantization_mode in QUANTIZATION_CONFIG:
        info = QUANTIZATION_CONFIG[quantization_mode]
        print(f"Mode: {quantization_mode.upper()}")
        print(f"Description: {info['description']}")
        print(f"Speed Boost: {info['speed_boost']}")
        print(f"Memory Saving: {info['memory_saving']}")
        print(f"Accuracy Loss: {info['accuracy_loss']}")
        print(f"Cost Reduction: {info['cost_reduction']}")
        
        # Calculate new training time and cost
        if PRODUCTION_MODE:
            base_time = 12  # hours
            base_cost = 15   # dollars
            
            speed_multiplier = float(info['speed_boost'].replace('x', ''))
            cost_reduction = float(info['cost_reduction'].split('-')[0]) / 100
            
            new_time = base_time / speed_multiplier
            new_cost = base_cost * (1 - cost_reduction)
            
            print(f"\n💰 BUDGET IMPACT:")
            print(f"   Original Time: {base_time}h → {new_time:.1f}h")
            print(f"   Original Cost: ${base_cost} → ${new_cost:.1f}")
            print(f"   Savings: ${base_cost - new_cost:.1f} ({info['cost_reduction']})")
    
    print("="*60)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageNet
import os
import time
import math
import numpy as np
from tqdm import tqdm
import json
from collections import OrderedDict
import wandb
from accelerate import Accelerator
from transformers import AutoModel, AutoTokenizer
import warnings
import logging
import psutil
import gc
from datetime import datetime
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

# =============================================================================
# 📊 COMPREHENSIVE LOGGING AND MONITORING SYSTEM
# =============================================================================

class TrainingMonitor:
    """
    Comprehensive training monitoring and logging system
    """
    def __init__(self, log_dir='logs', enable_wandb=False):
        self.log_dir = log_dir
        self.enable_wandb = enable_wandb
        self.start_time = time.time()
        self.epoch_times = []
        self.memory_usage = []
        self.gpu_memory_usage = []
        self.losses = []
        self.accuracies = []
        self.learning_rates = []
        
        # Setup logging
        os.makedirs(log_dir, exist_ok=True)
        self.setup_logging()
        
        # Initialize monitoring
        self.log_system_info()
        
    def setup_logging(self):
        """Setup comprehensive logging system"""
        log_file = os.path.join(self.log_dir, f'training_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
        
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
        self.logger.info("🚀 Training monitoring system initialized")
        
    def log_system_info(self):
        """Log system information"""
        self.logger.info("=" * 60)
        self.logger.info("🖥️ SYSTEM INFORMATION")
        self.logger.info("=" * 60)
        self.logger.info(f"Device: {device}")
        self.logger.info(f"CUDA Available: {torch.cuda.is_available()}")
        
        if torch.cuda.is_available():
            self.logger.info(f"GPU: {torch.cuda.get_device_name(0)}")
            self.logger.info(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
            self.logger.info(f"CUDA Version: {torch.version.cuda}")
        
        self.logger.info(f"PyTorch Version: {torch.__version__}")
        self.logger.info(f"CPU Count: {psutil.cpu_count()}")
        self.logger.info(f"RAM: {psutil.virtual_memory().total / 1e9:.1f} GB")
        self.logger.info("=" * 60)
        
    def log_epoch_start(self, epoch, total_epochs):
        """Log epoch start information"""
        self.logger.info(f"🚀 Starting Epoch {epoch}/{total_epochs}")
        self.logger.info(f"   Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
    def log_epoch_end(self, epoch, train_loss, train_acc, val_loss, val_acc, lr, epoch_time):
        """Log epoch end information"""
        self.epoch_times.append(epoch_time)
        self.losses.append({'train': train_loss, 'val': val_loss})
        self.accuracies.append({'train': train_acc, 'val': val_acc})
        self.learning_rates.append(lr)
        
        # Log epoch summary
        self.logger.info(f"✅ Epoch {epoch} completed in {epoch_time:.2f}s")
        self.logger.info(f"   Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        self.logger.info(f"   Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        self.logger.info(f"   Learning Rate: {lr:.6f}")
        
        # Log resource usage
        self.log_resource_usage()
        
    def log_resource_usage(self):
        """Log current resource usage"""
        # CPU and RAM usage
        cpu_percent = psutil.cpu_percent()
        ram = psutil.virtual_memory()
        ram_percent = ram.percent
        ram_used_gb = ram.used / 1e9
        
        self.logger.info(f"📊 Resource Usage:")
        self.logger.info(f"   CPU: {cpu_percent:.1f}%")
        self.logger.info(f"   RAM: {ram_percent:.1f}% ({ram_used_gb:.1f} GB used)")
        
        # GPU usage
        if torch.cuda.is_available():
            gpu_memory = torch.cuda.memory_allocated() / 1e9
            gpu_memory_max = torch.cuda.max_memory_allocated() / 1e9
            gpu_utilization = torch.cuda.utilization() if hasattr(torch.cuda, 'utilization') else 0
            
            self.logger.info(f"   GPU Memory: {gpu_memory:.1f} GB (Peak: {gpu_memory_max:.1f} GB)")
            if gpu_utilization > 0:
                self.logger.info(f"   GPU Utilization: {gpu_utilization:.1f}%")
        
        # Store for tracking
        self.memory_usage.append({
            'cpu_percent': cpu_percent,
            'ram_percent': ram_percent,
            'ram_used_gb': ram_used_gb,
            'gpu_memory_gb': gpu_memory if torch.cuda.is_available() else 0
        })
        
    def log_training_complete(self, best_accuracy, total_time):
        """Log training completion"""
        self.logger.info("=" * 60)
        self.logger.info("🎉 TRAINING COMPLETED")
        self.logger.info("=" * 60)
        self.logger.info(f"Total Time: {total_time/3600:.2f} hours")
        self.logger.info(f"Best Accuracy: {best_accuracy:.2f}%")
        self.logger.info(f"Average Epoch Time: {np.mean(self.epoch_times):.2f}s")
        self.logger.info(f"Total Epochs: {len(self.epoch_times)}")
        
        # Log final resource usage
        self.log_resource_usage()
        
        # Save training summary
        self.save_training_summary(best_accuracy, total_time)
        
    def save_training_summary(self, best_accuracy, total_time):
        """Save comprehensive training summary"""
        summary = {
            'training_info': {
                'start_time': self.start_time,
                'total_time_hours': total_time / 3600,
                'best_accuracy': best_accuracy,
                'total_epochs': len(self.epoch_times),
                'average_epoch_time': np.mean(self.epoch_times)
            },
            'performance_metrics': {
                'losses': self.losses,
                'accuracies': self.accuracies,
                'learning_rates': self.learning_rates,
                'epoch_times': self.epoch_times
            },
            'resource_usage': self.memory_usage,
            'system_info': {
                'device': str(device),
                'cuda_available': torch.cuda.is_available(),
                'pytorch_version': torch.__version__
            }
        }
        
        summary_file = os.path.join(self.log_dir, 'training_summary.json')
        with open(summary_file, 'w') as f:
            json.dump(summary, f, indent=2)
        
        self.logger.info(f"📄 Training summary saved to: {summary_file}")

# NOTE: The TrainingMonitor is defined here, but instantiated after the config is created.

# =============================================================================
# 🛑 EARLY STOPPING MECHANISM
# =============================================================================

class EarlyStopping:
    """
    Early stopping mechanism to prevent overfitting and save resources
    """
    def __init__(self, patience=10, min_delta=0.001, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_score = None
        self.counter = 0
        self.best_weights = None
        self.early_stop = False
        
    def __call__(self, val_score, model):
        """
        Check if training should stop early
        Returns: True if training should stop, False otherwise
        """
        if self.best_score is None:
            self.best_score = val_score
            self.save_checkpoint(model)
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.restore_best_weights:
                    model.load_state_dict(self.best_weights)
                return True
        else:
            self.best_score = val_score
            self.counter = 0
            self.save_checkpoint(model)
        
        return False
    
    def save_checkpoint(self, model):
        """Save model weights"""
        self.best_weights = model.state_dict().copy()

# NOTE: early_stopping instance will be created after the config is defined.


## ResNet50 Implementation from Scratch


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1
    
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4
    
    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_planes = 64
        
        # FIXED: Modified for CIFAR-100 (32x32 images) vs ImageNet (224x224)
        if num_classes == 100:  # CIFAR-100
            # Smaller initial conv for CIFAR-100
            self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            self.bn1 = nn.BatchNorm2d(64)
            self.maxpool = nn.Identity()  # No maxpool for CIFAR-100
        else:  # ImageNet
            self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.bn1 = nn.BatchNorm2d(64)
            self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)  # Identity for CIFAR-100, MaxPool for ImageNet
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

def ResNet50(num_classes=1000):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes)

def ResNet18(num_classes=1000):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

def ResNet34(num_classes=1000):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes)

# Test the model
model = ResNet50()
print(f'Model parameters: {sum(p.numel() for p in model.parameters()):,}')
print(f'Model size: {sum(p.numel() for p in model.parameters()) * 4 / 1e6:.1f} MB')


## Data Loading and Preprocessing


In [ ]:
# Data augmentation and preprocessing
def get_transforms():
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.08, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
        transforms.RandomRotation(degrees=15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform

# For Colab testing, we'll use CIFAR-100 as a smaller dataset
# In production, replace with ImageNet
def get_cifar100_dataset():
    # FIXED: Proper transforms for CIFAR-100 (32x32 images)
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])  # CIFAR-100 stats
    ])
    
    val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])  # CIFAR-100 stats
    ])
    
    # Use CIFAR-100 for testing (100 classes, similar to ImageNet structure)
    train_dataset = torchvision.datasets.CIFAR100(
        root='./data', train=True, download=True, transform=train_transform
    )
    
    val_dataset = torchvision.datasets.CIFAR100(
        root='./data', train=False, download=True, transform=val_transform
    )
    
    return train_dataset, val_dataset

# For ImageNet (use this in production)
def get_imagenet_dataset(data_path):
    train_transform, val_transform = get_transforms()
    
    # Try torchvision ImageNet first; fallback to ImageFolder if index not found
    try:
        train_dataset = ImageNet(
            root=data_path, split='train', transform=train_transform
        )
        val_dataset = ImageNet(
            root=data_path, split='val', transform=val_transform
        )
        print("✅ Loaded torchvision.datasets.ImageNet")
    except Exception as e:
        print(f"⚠️ torchvision.datasets.ImageNet failed: {e}")
        print("   Falling back to torchvision.datasets.ImageFolder (expects class subfolders)...")
        from torchvision.datasets import ImageFolder
        train_dataset = ImageFolder(
            root=os.path.join(data_path, 'train'), transform=train_transform
        )
        val_dataset = ImageFolder(
            root=os.path.join(data_path, 'val'), transform=val_transform
        )
        print("✅ Loaded torchvision.datasets.ImageFolder")
    
    return train_dataset, val_dataset

# =============================================================================
# 📁 DATASET LOADING WITH VALIDATION
# =============================================================================

def validate_dataset(dataset, dataset_name):
    """
    Comprehensive dataset validation
    """
    print(f"🔍 Validating {dataset_name} dataset...")
    
    # Check dataset size
    if len(dataset) == 0:
        raise ValueError(f"{dataset_name} dataset is empty!")
    
    # Check for corrupted samples
    corrupted_samples = 0
    valid_samples = 0
    
    for i in range(min(100, len(dataset))):  # Check first 100 samples
        try:
            data, target = dataset[i]
            if torch.isnan(data).any() or torch.isinf(data).any():
                corrupted_samples += 1
            else:
                valid_samples += 1
        except Exception as e:
            corrupted_samples += 1
    
    print(f"   Dataset size: {len(dataset)}")
    print(f"   Valid samples (checked): {valid_samples}")
    print(f"   Corrupted samples: {corrupted_samples}")
    
    if corrupted_samples > len(dataset) * 0.1:  # More than 10% corrupted
        print(f"⚠️ WARNING: High corruption rate in {dataset_name} ({corrupted_samples/len(dataset)*100:.1f}%)")
    
    return valid_samples > 0

if TESTING_MODE:
    # 🧪 TESTING: Use CIFAR-100 (smaller dataset for Colab)
    print("Loading CIFAR-100 dataset for testing...")
    train_dataset, val_dataset = get_cifar100_dataset()
    batch_size = 16  # Smaller batch for Colab memory
    num_workers = 2  # Fewer workers for Colab
    print(f"✅ CIFAR-100 loaded: {len(train_dataset)} train, {len(val_dataset)} val samples")
    print(f"✅ Number of classes: {len(train_dataset.classes)}")
    
    # Validate datasets
    validate_dataset(train_dataset, "CIFAR-100 Train")
    validate_dataset(val_dataset, "CIFAR-100 Val")
    
elif PRODUCTION_MODE:
    # 🏭 PRODUCTION: Use ImageNet-1K (full dataset)
    print("Loading ImageNet-1K dataset for production...")
    train_dataset, val_dataset = get_imagenet_dataset('./imagenet/')
    batch_size = 64  # Larger batch for production
    num_workers = 8  # More workers for production
    print(f"✅ ImageNet-1K loaded: {len(train_dataset)} train, {len(val_dataset)} val samples")
    print(f"✅ Number of classes: {len(train_dataset.classes)}")
    
    # Validate datasets
    validate_dataset(train_dataset, "ImageNet Train")
    validate_dataset(val_dataset, "ImageNet Val")

print(f"Batch size: {batch_size}")
print(f"Number of workers: {num_workers}")

# Create data loaders with error handling
try:
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, 
        num_workers=num_workers, pin_memory=True, 
        persistent_workers=True if num_workers > 0 else False
    )
    
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, 
        num_workers=num_workers, pin_memory=True,
        persistent_workers=True if num_workers > 0 else False
    )
    
    print("✅ Data loaders created successfully")
    
except Exception as e:
    print(f"⚠️ WARNING: Error creating data loaders: {e}")
    print("   Falling back to single-threaded loading...")
    
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, 
        num_workers=0, pin_memory=False
    )
    
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False, 
        num_workers=0, pin_memory=False
    )

print(f'Train samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')
print(f'Number of classes: {len(train_dataset.classes)}')


## Training Configuration and Optimizations


In [ ]:
# =============================================================================
# ⚙️ TRAINING CONFIGURATION BASED ON MODE
# =============================================================================

if TESTING_MODE:
    # 🧪 TESTING CONFIGURATION (Colab-friendly)
    config = {
        'epochs': 5,  # Reduced for quick testing
        'learning_rate': 0.05,  # FIXED: Proper learning rate for CIFAR-100 (0.1 was too high!)
        'weight_decay': 1e-4,
        'momentum': 0.9,
        'batch_size': batch_size,
        'num_classes': len(train_dataset.classes),
        'save_every': 2,  # Save more frequently for testing
        'eval_every': 1,  # Evaluate every epoch
        'mixed_precision': QUANTIZATION_MODE == "fp16",  # Enable based on quantization
        'gradient_accumulation_steps': 1,
        'warmup_epochs': 0,  # FIXED: No warmup for CIFAR-100 (causes issues)
        'cosine_annealing': True,
        'target_accuracy': 80.0,  # Higher target for CIFAR-100
        'wandb_enabled': False,  # Disable wandb for testing
        'quantization_mode': QUANTIZATION_MODE,  # Add quantization mode
        'quantization_enabled': QUANTIZATION_MODE != "none",
        'torch_compile': True,  # Enable torch.compile optimization
        'compile_mode': 'default',  # 'default', 'reduce-overhead', 'max-autotune'
        # Enhanced monitoring and error handling
        'early_stopping_patience': 3,  # Early stopping for testing
        'early_stopping_min_delta': 0.001,  # Minimum improvement threshold
        'gradient_clip_norm': 1.0,  # Gradient clipping
        'memory_cleanup_frequency': 5  # Memory cleanup every N epochs
    }
    
elif PRODUCTION_MODE:
    # 🏭 PRODUCTION CONFIGURATION (Full ImageNet training)
    config = {
        'epochs': 90,  # Full training
        'learning_rate': 0.1,  # Standard for ImageNet with proper scheduling
        'weight_decay': 1e-4,
        'momentum': 0.9,
        'batch_size': batch_size,
        'num_classes': len(train_dataset.classes),
        'save_every': 10,  # Save every 10 epochs
        'eval_every': 5,  # Evaluate every 5 epochs
        'mixed_precision': QUANTIZATION_MODE == "fp16",  # Enable based on quantization
        'gradient_accumulation_steps': 1,
        'warmup_epochs': 5,  # Full warmup
        'cosine_annealing': True,
        'target_accuracy': 75.0,  # ImageNet target
        'wandb_enabled': True,  # Enable wandb for production
        'quantization_mode': QUANTIZATION_MODE,  # Add quantization mode
        'quantization_enabled': QUANTIZATION_MODE != "none",
        'torch_compile': True,  # Enable torch.compile optimization
        'compile_mode': 'default',  # 'default', 'reduce-overhead', 'max-autotune'
        # Enhanced monitoring and error handling
        'early_stopping_patience': 10,  # Early stopping for production
        'early_stopping_min_delta': 0.001,  # Minimum improvement threshold
        'gradient_clip_norm': 1.0,  # Gradient clipping
        'memory_cleanup_frequency': 10  # Memory cleanup every N epochs
    }

print("Training Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

# Initialize model for the correct number of classes
# FIXED: Use ResNet18 for CIFAR-100 (ResNet50 is too deep for 32x32 images)
if TESTING_MODE:
    model = ResNet18(num_classes=config['num_classes'])  # ResNet18 for CIFAR-100
    print("🏗️ Using ResNet18 for CIFAR-100 (32x32 images)")
else:
    model = ResNet50(num_classes=config['num_classes'])  # ResNet50 for ImageNet
    print("🏗️ Using ResNet50 for ImageNet (224x224 images)")

model = model.to(device)

# FIXED: Proper initialization for CIFAR-100
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)

# Apply proper initialization
model.apply(init_weights)
print(f"✅ Model initialized with proper weights for {config['num_classes']} classes")

# DEBUG: Test model with a sample batch
print("\n🔍 DEBUG - Testing model with sample batch...")
model.eval()
with torch.no_grad():
    sample_data, sample_target = next(iter(train_loader))
    sample_data = sample_data[:4].to(device)  # FIXED: Move to same device as model
    sample_target = sample_target[:4].to(device)  # FIXED: Move to same device as model
    
    print(f"   Input shape: {sample_data.shape}")
    print(f"   Target shape: {sample_target.shape}")
    print(f"   Target values: {sample_target.tolist()}")
    print(f"   Device: {sample_data.device} (model: {next(model.parameters()).device})")
    
    sample_output = model(sample_data)
    print(f"   Output shape: {sample_output.shape}")
    print(f"   Output range: [{sample_output.min().item():.3f}, {sample_output.max().item():.3f}]")
    
    sample_pred = sample_output.argmax(dim=1)
    print(f"   Predictions: {sample_pred.tolist()}")
    
    sample_acc = (sample_pred == sample_target).float().mean().item()
    print(f"   Sample accuracy: {sample_acc:.2f}%")
    
    # Check if model is learning (outputs should be reasonable)
    if sample_output.std().item() < 0.1:
        print("⚠️ WARNING: Model outputs have very low variance - may indicate initialization issues")
    else:
        print("✅ Model outputs have reasonable variance")

model.train()  # Set back to training mode

# =============================================================================
# 🔍 VERIFY MODEL CONFIGURATION
# =============================================================================

print(f"\n🔍 Model Configuration Verification:")
print(f"   Dataset: {'CIFAR-100' if TESTING_MODE else 'ImageNet-1K'}")
print(f"   Number of classes: {config['num_classes']}")
print(f"   Model output size: {model.fc.out_features}")
print(f"   Target accuracy: {config['target_accuracy']}%")
print(f"   Learning rate: {config['learning_rate']}")

# Verify model output matches dataset classes
if model.fc.out_features == config['num_classes']:
    print("✅ Model output size matches dataset classes")
else:
    print(f"❌ MISMATCH: Model outputs {model.fc.out_features} classes, dataset has {config['num_classes']} classes")

# =============================================================================
# 🚀 TORCH.COMPILE OPTIMIZATION (10% speed boost)
# =============================================================================

# Apply torch.compile for 10% speed boost (PyTorch 2.0+)
if config.get('torch_compile', False):
    if hasattr(torch, 'compile'):
        try:
            compile_mode = config.get('compile_mode', 'default')
            print(f"🚀 Applying torch.compile() optimization with mode: {compile_mode}")
            print("   Note: First few iterations will be slower due to compilation...")
            model = torch.compile(model, mode=compile_mode)
            print("✅ Model compiled successfully! Expected 10% speed boost.")
        except Exception as e:
            print(f"⚠️ torch.compile() failed: {e}")
            print("   Continuing without compilation...")
    else:
        print("⚠️ torch.compile() not available (requires PyTorch 2.0+)")
        print("   Continuing without compilation...")
else:
    print("📊 torch.compile() disabled in configuration")

# Initialize accelerator for mixed precision training
accelerator = Accelerator(mixed_precision='fp16' if config['mixed_precision'] else 'no')

# Optimizer with learning rate scheduling
optimizer = optim.SGD(
    model.parameters(), 
    lr=config['learning_rate'], 
    momentum=config['momentum'], 
    weight_decay=config['weight_decay']
)

# Learning rate scheduler
def get_lr_scheduler(optimizer, num_epochs, warmup_epochs=5):
    def lr_lambda(epoch):
        if warmup_epochs == 0:
            # No warmup - use cosine annealing directly
            return 0.5 * (1 + math.cos(math.pi * epoch / num_epochs))
        elif epoch < warmup_epochs:
            return epoch / warmup_epochs
        else:
            return 0.5 * (1 + math.cos(math.pi * (epoch - warmup_epochs) / (num_epochs - warmup_epochs)))
    
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

scheduler = get_lr_scheduler(optimizer, config['epochs'], config['warmup_epochs'])

# Loss function
# FIXED: Reduce label smoothing for CIFAR-100 (0.1 is too high for small dataset)
if TESTING_MODE:
    criterion = nn.CrossEntropyLoss(label_smoothing=0.0)  # No label smoothing for CIFAR-100
    print("🎯 Using CrossEntropyLoss without label smoothing for CIFAR-100")
else:
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Label smoothing for ImageNet
    print("🎯 Using CrossEntropyLoss with label smoothing for ImageNet")

# Prepare for accelerator
model, optimizer, train_loader, val_loader, scheduler = accelerator.prepare(
    model, optimizer, train_loader, val_loader, scheduler
)

# Instantiate monitoring and early stopping now that config exists
monitor = TrainingMonitor(log_dir='logs', enable_wandb=config.get('wandb_enabled', False))
early_stopping = EarlyStopping(
    patience=config.get('early_stopping_patience', 10),
    min_delta=config.get('early_stopping_min_delta', 0.001),
    restore_best_weights=True
)

print(f"Model prepared for training on {device}")
print(f"Mixed precision: {config['mixed_precision']}")


## Training and Evaluation Functions


In [ ]:
# =============================================================================
# 🚀 APPLY QUANTIZATION FOR SPEED & BUDGET OPTIMIZATION
# =============================================================================

# Apply quantization if enabled
if config['quantization_enabled']:
    print(f"\n🔧 Applying {config['quantization_mode']} quantization...")
    model = apply_quantization(model, config['quantization_mode'], device)
    
    # Print quantization benefits
    quant_info = get_quantization_info(model, config['quantization_mode'])
    print(f"\n📊 Model Info with {config['quantization_mode']} quantization:")
    print(f"   Model Size: {quant_info['model_size_mb']:.1f} MB")
    print(f"   Parameters: {quant_info['parameters']:,}")
    print(f"   Speed Boost: {quant_info['speed_boost']}")
    print(f"   Memory Saving: {quant_info['memory_saving']}")
    
    # Print budget impact for production
    if PRODUCTION_MODE:
        print_quantization_summary(config['quantization_mode'], quant_info)
else:
    print("📊 No quantization applied - using full precision")
    print(f"   Model Size: {sum(p.numel() for p in model.parameters()) * 4 / 1e6:.1f} MB")
    print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")


In [ ]:
def train_epoch(model, train_loader, optimizer, criterion, accelerator, epoch):
    """
    Enhanced training epoch with comprehensive error handling and monitoring
    """
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    failed_batches = 0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch}')
    
    for batch_idx, (data, target) in enumerate(pbar):
        try:
            # FIXED: Ensure data and target are on the same device as model
            data = data.to(device, non_blocking=True)
            target = target.to(device, non_blocking=True)
            
            # Data validation
            if data.isnan().any() or target.isnan().any():
                print(f"⚠️ WARNING: NaN detected in batch {batch_idx}, skipping...")
                failed_batches += 1
                continue
                
            if data.isinf().any() or target.isinf().any():
                print(f"⚠️ WARNING: Inf detected in batch {batch_idx}, skipping...")
                failed_batches += 1
                continue
            
            with accelerator.accumulate(model):
                optimizer.zero_grad()
                
                output = model(data)
                loss = criterion(output, target)
                
                # Check for NaN loss
                if torch.isnan(loss) or torch.isinf(loss):
                    print(f"⚠️ WARNING: Invalid loss detected in batch {batch_idx}, skipping...")
                    failed_batches += 1
                    continue
                
                accelerator.backward(loss)
                
                # Gradient clipping to prevent exploding gradients
                if hasattr(accelerator, 'clip_grad_norm_'):
                    accelerator.clip_grad_norm_(
                        model.parameters(), max_norm=config.get('gradient_clip_norm', 1.0)
                    )
                
                optimizer.step()
                
                total_loss += loss.item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
                total += target.size(0)
                
                # DEBUG: Print first batch accuracy for verification
                if batch_idx == 0:
                    batch_acc = 100. * pred.eq(target.view_as(pred)).sum().item() / target.size(0)
                    print(f"\n🔍 DEBUG - First batch accuracy: {batch_acc:.2f}%")
                    print(f"   Batch size: {target.size(0)}")
                    print(f"   Correct predictions: {pred.eq(target.view_as(pred)).sum().item()}")
                    print(f"   Sample predictions: {pred[:5].flatten().tolist()}")
                    print(f"   Sample targets: {target[:5].tolist()}")
                    print(f"   Device check: data={data.device}, target={target.device}, model={next(model.parameters()).device}")
                
                # Update progress bar
                pbar.set_postfix({
                    'Loss': f'{loss.item():.4f}',
                    'Acc': f'{100. * correct / total:.2f}%',
                    'LR': f'{optimizer.param_groups[0]["lr"]:.6f}',
                    'Failed': f'{failed_batches}'
                })
                
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print(f"🚨 CUDA OOM ERROR in batch {batch_idx}: {e}")
                print("   Attempting recovery...")
                
                # Clear cache and try again with smaller batch
                torch.cuda.empty_cache()
                if hasattr(torch.cuda, 'reset_peak_memory_stats'):
                    torch.cuda.reset_peak_memory_stats()
                
                # Reduce batch size for next iteration
                if batch_size > 1:
                    print(f"   Reducing batch size from {batch_size} to {max(1, batch_size // 2)}")
                    # Note: This would require recreating the dataloader
                
                failed_batches += 1
                continue
            else:
                print(f"🚨 RUNTIME ERROR in batch {batch_idx}: {e}")
                failed_batches += 1
                continue
                
        except Exception as e:
            print(f"🚨 UNEXPECTED ERROR in batch {batch_idx}: {e}")
            failed_batches += 1
            continue
    
    # Report failed batches
    if failed_batches > 0:
        print(f"⚠️ WARNING: {failed_batches} batches failed in epoch {epoch}")
    
    avg_loss = total_loss / len(train_loader) if len(train_loader) > 0 else 0
    accuracy = 100. * correct / total if total > 0 else 0
    
    return avg_loss, accuracy

def evaluate(model, val_loader, criterion, accelerator):
    """
    Enhanced evaluation with comprehensive error handling
    """
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    failed_batches = 0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(tqdm(val_loader, desc='Evaluating')):
            try:
                # FIXED: Ensure data and target are on the same device as model
                data = data.to(device, non_blocking=True)
                target = target.to(device, non_blocking=True)
                
                # Data validation
                if data.isnan().any() or target.isnan().any():
                    print(f"⚠️ WARNING: NaN detected in validation batch {batch_idx}, skipping...")
                    failed_batches += 1
                    continue
                    
                if data.isinf().any() or target.isinf().any():
                    print(f"⚠️ WARNING: Inf detected in validation batch {batch_idx}, skipping...")
                    failed_batches += 1
                    continue
                
                output = model(data)
                loss = criterion(output, target)
                
                # Check for NaN loss
                if torch.isnan(loss) or torch.isinf(loss):
                    print(f"⚠️ WARNING: Invalid loss detected in validation batch {batch_idx}, skipping...")
                    failed_batches += 1
                    continue
                
                total_loss += loss.item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
                total += target.size(0)
                
            except RuntimeError as e:
                if "out of memory" in str(e).lower():
                    print(f"🚨 CUDA OOM ERROR in validation batch {batch_idx}: {e}")
                    torch.cuda.empty_cache()
                    failed_batches += 1
                    continue
                else:
                    print(f"🚨 RUNTIME ERROR in validation batch {batch_idx}: {e}")
                    failed_batches += 1
                    continue
                    
            except Exception as e:
                print(f"🚨 UNEXPECTED ERROR in validation batch {batch_idx}: {e}")
                failed_batches += 1
                continue
    
    # Report failed batches
    if failed_batches > 0:
        print(f"⚠️ WARNING: {failed_batches} validation batches failed")
    
    avg_loss = total_loss / len(val_loader) if len(val_loader) > 0 else 0
    accuracy = 100. * correct / total if total > 0 else 0
    
    return avg_loss, accuracy

def save_checkpoint(model, optimizer, scheduler, epoch, accuracy, filepath):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'accuracy': accuracy,
        'config': config
    }
    torch.save(checkpoint, filepath)
    print(f'Checkpoint saved: {filepath}')

def load_checkpoint(filepath, model, optimizer, scheduler):
    checkpoint = torch.load(filepath, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return checkpoint['epoch'], checkpoint['accuracy']


## Main Training Loop


In [ ]:
# =============================================================================
# 📊 LOGGING SETUP BASED ON MODE
# =============================================================================

if config['wandb_enabled']:
    print("Initializing Weights & Biases for logging...")
    wandb.init(project="imagenet-resnet50", config=config)
else:
    print("Wandb logging disabled for testing mode")

# Create output directory
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('logs', exist_ok=True)

# =============================================================================
# 🚀 ENHANCED TRAINING LOOP WITH COMPREHENSIVE MONITORING
# =============================================================================

# Training loop with enhanced monitoring and error handling
best_accuracy = 0
start_time = time.time()
patience_counter = 0

print("🚀 Starting enhanced training with comprehensive monitoring...")
print(f"Target accuracy: {config['target_accuracy']}%")
print(f"Budget constraint: $25")
print(f"Training for {config['epochs']} epochs")
print(f"Early stopping patience: {config.get('early_stopping_patience', 10)}")

# Log training start
monitor.logger.info("🚀 TRAINING STARTED")
monitor.logger.info(f"Target accuracy: {config['target_accuracy']}%")
monitor.logger.info(f"Total epochs: {config['epochs']}")
monitor.logger.info(f"Early stopping patience: {config.get('early_stopping_patience', 10)}")

for epoch in range(1, config['epochs'] + 1):
    try:
        epoch_start = time.time()
        
        # Log epoch start
        monitor.log_epoch_start(epoch, config['epochs'])
        
        # Train with error handling
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, accelerator, epoch)
        
        # Update learning rate
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        
        # Evaluate with error handling
        if epoch % config['eval_every'] == 0 or epoch == config['epochs']:
            val_loss, val_acc = evaluate(model, val_loader, criterion, accelerator)
            
            # Log epoch results
            epoch_time = time.time() - epoch_start
            monitor.log_epoch_end(epoch, train_loss, train_acc, val_loss, val_acc, current_lr, epoch_time)
            
            print(f'\n📊 Epoch {epoch} Results:')
            print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
            print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
            print(f'  Learning Rate: {current_lr:.6f}')
            print(f'  Epoch Time: {epoch_time:.2f}s')
            
            # Save best model
            if val_acc > best_accuracy:
                best_accuracy = val_acc
                save_checkpoint(
                    model, optimizer, scheduler, epoch, val_acc,
                    f'checkpoints/best_model_epoch_{epoch}_acc_{val_acc:.2f}.pth'
                )
                monitor.logger.info(f"🏆 New best model saved with accuracy: {val_acc:.2f}%")
            
            # Early stopping check
            if early_stopping(val_acc, model):
                monitor.logger.info(f"🛑 Early stopping triggered at epoch {epoch}")
                print(f'\n🛑 Early stopping triggered at epoch {epoch}')
                print(f'Best accuracy achieved: {best_accuracy:.2f}%')
                break
            
            # Log to wandb (if enabled)
            if config['wandb_enabled']:
                wandb.log({
                    'epoch': epoch,
                    'train_loss': train_loss,
                    'train_acc': train_acc,
                    'val_loss': val_loss,
                    'val_acc': val_acc,
                    'learning_rate': current_lr,
                    'epoch_time': epoch_time
                })
        else:
            # Log epoch without validation
            epoch_time = time.time() - epoch_start
            monitor.logger.info(f"✅ Epoch {epoch} completed in {epoch_time:.2f}s (no validation)")
            print(f'  Epoch time: {epoch_time:.2f}s')
        
        # Save checkpoint every N epochs
        if epoch % config['save_every'] == 0:
            save_checkpoint(
                model, optimizer, scheduler, epoch, val_acc if 'val_acc' in locals() else 0,
                f'checkpoints/checkpoint_epoch_{epoch}.pth'
            )
            monitor.logger.info(f"💾 Checkpoint saved at epoch {epoch}")
        
        # Check if we've reached target accuracy
        if 'val_acc' in locals() and val_acc >= config['target_accuracy']:
            monitor.logger.info(f"🎉 Target accuracy of {config['target_accuracy']}% reached!")
            print(f'\n🎉 Target accuracy of {config["target_accuracy"]}% reached! Stopping early.')
            break
            
        # Memory cleanup
        if epoch % config.get('memory_cleanup_frequency', 5) == 0:
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                
    except Exception as e:
        monitor.logger.error(f"🚨 CRITICAL ERROR in epoch {epoch}: {e}")
        print(f'\n🚨 CRITICAL ERROR in epoch {epoch}: {e}')
        print("Attempting to continue training...")
        
        # Try to recover
        try:
            torch.cuda.empty_cache()
            gc.collect()
        except:
            pass
        
        # Continue with next epoch
        continue

# Training completion
total_time = time.time() - start_time
monitor.log_training_complete(best_accuracy, total_time)

print(f'\n🎉 Training completed in {total_time/3600:.2f} hours')
print(f'Best accuracy: {best_accuracy:.2f}%')

# Save final model
save_checkpoint(
    model, optimizer, scheduler, config['epochs'], best_accuracy,
    'checkpoints/final_model.pth'
)

print("📄 Training summary and logs saved to 'logs/' directory")
print("🚀 Enhanced training with comprehensive monitoring completed!")


## Model Evaluation and Testing


In [ ]:
# Final evaluation
print("\nFinal evaluation...")
final_loss, final_acc = evaluate(model, val_loader, criterion, accelerator)
print(f'Final validation accuracy: {final_acc:.2f}%')
print(f'Final validation loss: {final_loss:.4f}')

# Test on a few samples
model.eval()
with torch.no_grad():
    for i, (data, target) in enumerate(val_loader):
        if i >= 3:  # Test on first 3 batches
            break
        
        output = model(data)
        pred = output.argmax(dim=1)
        
        print(f'\nBatch {i+1}:')
        for j in range(min(5, len(data))):
            true_class = train_dataset.classes[target[j].item()]
            pred_class = train_dataset.classes[pred[j].item()]
            confidence = F.softmax(output[j], dim=0)[pred[j]].item()
            
            print(f'  True: {true_class}, Pred: {pred_class}, Conf: {confidence:.3f}')


## Cost Estimation and Budget Optimization


In [ ]:
# Cost estimation for EC2 training
def estimate_training_cost():
    # EC2 instance costs (as of 2024)
    instance_costs = {
        'g4dn.xlarge': 0.526,  # 1 GPU, 4 vCPU, 16 GB RAM
        'g4dn.2xlarge': 0.752,  # 1 GPU, 8 vCPU, 32 GB RAM
        'g4dn.4xlarge': 1.204,  # 1 GPU, 16 vCPU, 64 GB RAM
        'p3.2xlarge': 3.06,     # 1 V100 GPU
        'p3.8xlarge': 12.24,    # 4 V100 GPUs
    }
    
    # Estimated training time (hours)
    estimated_hours = {
        'g4dn.xlarge': 24,      # Slower training
        'g4dn.2xlarge': 18,    # Medium speed
        'g4dn.4xlarge': 12,    # Faster training
        'p3.2xlarge': 8,        # V100 is much faster
        'p3.8xlarge': 4,        # Multiple V100s
    }
    
    print("EC2 Training Cost Estimation:")
    print("=" * 50)
    
    for instance_type in instance_costs:
        hourly_cost = instance_costs[instance_type]
        training_hours = estimated_hours[instance_type]
        total_cost = hourly_cost * training_hours
        
        print(f"{instance_type:12} | ${hourly_cost:6.3f}/hr | {training_hours:2d}hrs | ${total_cost:6.2f} total")
        
        if total_cost <= 25:
            print(f"  ✅ Within budget!")
        else:
            print(f"  ❌ Over budget")
    
    print("\nRecommended for $25 budget:")
    print("1. g4dn.2xlarge (18 hours) - $13.54")
    print("2. g4dn.4xlarge (12 hours) - $14.45")
    print("3. p3.2xlarge (8 hours) - $24.48")
    
    print("\nOptimization strategies:")
    print("- Use mixed precision training (fp16)")
    print("- Implement gradient accumulation")
    print("- Use efficient data loading")
    print("- Early stopping when target accuracy reached")
    print("- Use smaller batch sizes if memory constrained")

estimate_training_cost()
